In [1]:
import sys

sys.path.append('../../..')

1) Prepare the dataset in the pytorch Dataset format.
For example like this:

In [2]:
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.datasets import ImageFolder

train_dataset_path = '' # your path to train part of dataset
val_dataset_path = '' # your path to validation part of dataset

transform = Compose([ToTensor(), Resize((256, 256))])
train_dataset = ImageFolder(root=train_dataset_path, transform=transform)
val_dataset = ImageFolder(root=val_dataset_path, transform=transform)

ImageFolder is a generic data loader where the images are arranged in this way by default:

root/dog/xxx.png
root/dog/xxy.png
root/dog/[...]/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/[...]/asd932_.png

2) Model initialization.

In [3]:
from torchvision.models import resnet18
from core.architecture.experiment.nn_experimenter import ClassificationExperimenter

experimenter = ClassificationExperimenter(
    model=resnet18(num_classes=21), # Number of classes in your dataset
    name='ResNet18'
)

3) Fit parameters.

In [ ]:
from torch.utils.data import DataLoader
from core.architecture.experiment.nn_experimenter import FitParameters

dl_params={'batch_size': 32, 'num_workers': 8}

fit_params = FitParameters(
    dataset_name='Land-Use_Scene_Classification',
    train_dl=DataLoader(train_dataset, shuffle=True, **dl_params),
    val_dl=DataLoader(val_dataset, **dl_params),
    num_epochs=50
)

4) Initialization of the structure optimization (optionally).

In [ ]:
from core.operation.optimization.structure_optimization import SVDOptimization

svd_optimization = SVDOptimization(energy_thresholds=[0.1, 0.3, 0.5, 0.7, 0.9])

5) Running Model Training.
If you want to use structure optimization use ``svd_optimization.fit()`` instead of ``experimenter.fit()``

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import warnings
warnings.filterwarnings("ignore")

experimenter.fit(p=fit_params)
# svd_optimization.fit(exp=experimenter, params=fit_params)

Now you can get predictions for all images in a folder. The method returns a dictionary {'image_name': class}.

In [ ]:
from core.architecture.datasets.prediction_datasets import PredictionFolderDataset

prediction_dataset = PredictionFolderDataset(
    image_folder='', # Path to your folder with images
    transform=transform,
)
prediction_dl = DataLoader(prediction_dataset, **dl_params)
preds = experimenter.predict(prediction_dl)
preds